Instalación de bibliotecas necesarias para el correcto funcionamiento del Jupyter. Es importante asegurarse de que el kernel de Jupyter utilice la versión **3.11** de **Python** para garantizar la compatibilidad.

In [1]:
%pip install numpy pandas matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Entrenamiento

En esta sección se lleva a cabo el proceso completo de entrenamiento de un modelo **KNeighborsClassifier**. Se realiza una búsqueda de hiperparámetros para optimizar el modelo, seguida de un entrenamiento con todos los datos disponibles para esta fase. Finalmente, se prueba el modelo utilizando el conjunto de datos destinados para la validación y evaluación.

In [2]:
import pandas as pd
import numpy as np

from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

np.random.seed(42)

## Procesamiento de Datos

El primer paso es realizar un procesamiento básico de los datos de entrenamiento. En primer lugar, se elimina la columna correspondiente al ID del estudiante, ya que no aporta valor predictivo. Luego, se extrae la variable objetivo del conjunto de datos. Posteriormente, se escalan todas las características de entrada para asegurar que el modelo funcione correctamente. Finalmente, se codifica la variable objetivo, que en el dataset de entrenamiento se presenta como una variable cualitativa (label), para convertirla en una variable cuantitativa adecuada para el modelo.

Los datos de entrenamiento se dividen de la siguiente forma:

* 80% de los datos es para entrenar el modelo.
* 20% de los datos se usa para evaluar el modelo.

In [3]:
train = pd.read_csv("../storage/train.csv")
train

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [4]:
x = train.drop("Target", axis=1)
x = x.drop("id", axis=1)

y = train["Target"]

In [5]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
x_train, x_val, y_train, y_val = train_test_split(
    x, y_encoded, test_size=0.2, random_state=42, shuffle=True,
)

In [6]:
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)
x_val_sc = sc.transform(x_val)

## Búsqueda de Hiperparámetros

Con los datos correctamente preparados, se procede a la búsqueda de hiperparámetros utilizando el 50% del conjunto de datos de entrenamiento. Se exploran diferentes combinaciones de parámetros con el objetivo de identificar la configuración óptima para el modelo. Esta etapa es crucial para mejorar el rendimiento del modelo y garantizar que se ajuste adecuadamente a los datos.

Para la búsqueda de hiperparámetros, se configuró el parámetro `n_jobs` en -1, lo que permite utilizar las 16 CPUs disponibles en mi computadora para minimizar el tiempo de espera. Si prefieres no usar todos los recursos de tu máquina, puedes ajustar este parámetro según tu conveniencia.

In [7]:
x_train_sc_small, _, y_train_small, _ = train_test_split(
    x_train_sc, y_train, train_size=0.5, random_state=42, stratify=y_train,
)

param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [20, 30, 40],
    "p": [1, 2]
}

grid_search = GridSearchCV(
    KNeighborsClassifier(), param_grid, cv=5, scoring="accuracy", n_jobs=-1,
)
grid_search.fit(x_train_sc_small, y_train_small)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

score = best_model.score(x_val_sc, y_val)
print("Mejor Modelo:")
print(f"Modelo = {best_model}")
print(f"Parametros = {best_params}")
print(f"Score = {score}")

Mejor Modelo:
Modelo =  KNeighborsClassifier(leaf_size=20, n_neighbors=11, p=1)
Parametros =  {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
Score =  0.7948248823836905


## Entrenamiento Completo

Una vez identificada la mejor combinación de parámetros, se procede a entrenar el modelo utilizando el conjunto completo de datos de entrenamiento. Posteriormente, se realiza una evaluación del modelo para determinar su desempeño y asegurar que el entrenamiento haya sido efectivo.

In [22]:
model = KNeighborsClassifier(**best_params)
model.fit(x_train_sc, y_train)
score = model.score(x_val_sc, y_val)
print(f"Score = {score}")

Score = 0.7984840564558285


## Prueba

Con el modelo ya entrenado, se procede a realizar una prueba utilizando todo el conjunto de datos de prueba. Esta prueba tiene como objetivo observar las posibles predicciones del modelo y ofrecer un ejemplo de cómo realizar predicciones con el mismo. Como último paso del proceso de entrenamiento, se guarda el estandarizador de las variables (o escalador) y el modelo entrenado utilizando la biblioteca `joblib`.

In [23]:
x_test = pd.read_csv("../storage/test.csv")
x_test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [24]:
ids = x_test["id"]
x_test = x_test.drop("id", axis=1)
x_test_sc = sc.transform(x_test)

In [25]:
predictions = model.predict(x_test_sc)
pd.DataFrame({"id": ids, "prediction": predictions})

,id,prediction
0,76518,0
1,76519,2
2,76520,2
3,76521,2
4,76522,1
...,...,...
51007,127525,0
51008,127526,0
51009,127527,0
51010,127528,0


In [26]:
dump(model, "../storage/models/knn_fase_1.joblib")
dump(sc, "../storage/models/scaler_fase_1.joblib")

['../storage/models/scaler-fase-1.joblib']

# Predicción

En esta sección se carga el modelo previamente entrenado y se realizan predicciones utilizando el conjunto de datos de prueba. Un paso crucial es cargar el estandarizador de las variables (o escalador) para escalar el conjunto de datos de prueba, lo cual garantiza que el modelo funcione correctamente. El resultado de esta sección es un DataFrame que contiene todas las predicciones junto con el ID de los estudiantes.

In [27]:
import pandas as pd

from joblib import load

In [28]:
x_test = pd.read_csv("../storage/test.csv")
x_test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [29]:
model = load("../storage/models/knn_fase_1.joblib")
sc = load("../storage/models/scaler_fase_1.joblib")

In [30]:
ids = x_test["id"]
x_test = x_test.drop("id", axis=1)
x_test_sc = sc.transform(x_test)

In [31]:
predictions = model.predict(x_test_sc)
pd.DataFrame({"id": ids, "prediction": predictions})

,id,prediction
0,76518,0
1,76519,2
2,76520,2
3,76521,2
4,76522,1
...,...,...
51007,127525,0
51008,127526,0
51009,127527,0
51010,127528,0
